In [1]:
from dask_gateway import Gateway
from dask_gateway import GatewayCluster
from dask.distributed import Client

import socket
import pprint
import numpy
import pystare

import os
import sys

import boto3

In [23]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    done = False
    while not done:
        resp = s3_client.list_objects_v2(**kwargs)
        try:
            for obj in resp['Contents']:
                key = obj['Key']
                if key.startswith(prefix):
                    # print('key: ',key)
                    yield key
        except KeyError:
            print('Empty response from s3 for bucket %s with prefix %s'%(bucket,prefix))
            break

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
            


In [2]:
gateway = Gateway(); gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>

In [7]:
# gateway.close()

In [ ]:
# rezuma/jh:smce-mike-03

In [3]:
options = gateway.cluster_options(); options['image'] = 'rezuma/jh:smce-jdp-mike-03'; options

In [26]:
cluster = gateway.new_cluster(cluster_options=options); cluster

In [27]:
# cluster = gateway.connect('daskhub.e48268b909ab41d7a8e18fbb82a9a081'); cluster

In [71]:
cluster.

AttributeError: 'GatewayCluster' object has no attribute 'disconnect'

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/tornado/iostream.py", line 1417, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/home/jovyan/users_conda_envs/stare-1/lib/python3.7/ssl.py", line 1139, in do_handshake
    self._sslobj.do_handshake()
OSError: [Errno 0] Error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/tornado/iostream.py", line 711, in _handle_events
    self._handle_read()
  File "/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/tornado/iostream.py", 

In [9]:
if False:
    cluster.shutdown()

In [9]:
clusters = gateway.list_clusters(); clusters

[ClusterReport<name=daskhub.fd3a2fb9478a4264b86a60ccd840de8c, status=PENDING>,
 ClusterReport<name=daskhub.286d06b40fc247928b672c51d7b63d09, status=PENDING>]

In [5]:
cluster = gateway.connect(clusters[2].name)

In [6]:
cluster

In [7]:
cluster.shutdown()

In [7]:
# gateway_uri = "gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub."+gateway.list_clusters()[0].name.split('.')[1]; gateway_uri

In [29]:
# client = cluster.get_client(gateway_uri);
client = cluster.get_client();

/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/distributed/client.py:1136: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| python  | 3.7.8.final.0 | 3.7.7.final.0 | 3.7.7.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [16]:
# from dask.distributed import Client; client = Client()

In [30]:
client

Client Scheduler: gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.6f3d78cc545d4758a4e1c01d3c47c0fb Dashboard: /services/dask-gateway/clusters/daskhub.6f3d78cc545d4758a4e1c01d3c47c0fb/status,Cluster Workers: 1 Cores: 2 Memory: 4.29 GB


In [13]:
# client.close()

In [18]:
import dask.array as da
a = da.random.normal(size=(1000,1000),chunks=(50,50))
a.mean().compute()

0.0006699034249671197

In [31]:
client.nthreads()

{'tls://192.168.29.137:42757': 2}

In [32]:
# Dask helpers
def slam(client,action,data,partition_factor=1.5,dbg=0):
    np = sum(client.nthreads().values())
    if dbg>0: # a
        print('slam: np = %i'%np)
    shard_bounds = [int(i*len(data)/(1.0*partition_factor*np)) for i in range(int(partition_factor*np))] 
    if shard_bounds[-1] != len(data):
        if dbg>0: # b
            print('a sb[-1]: ',shard_bounds[-1],len(data))
        shard_bounds = shard_bounds + [len(data)]
        if dbg>0: # c
            print('sb: ',shard_bounds)
    data_shards = [data[shard_bounds[i]:shard_bounds[i+1]] for i in range(len(shard_bounds)-1)]
    if dbg>0: # d
        print('ds len:        ',len(data_shards))
        print('ds item len:   ',len(data_shards[0]))
        print('ds type:       ',type(data_shards[0]))
        try:
            print('ds dtype:      ',data_shards[0].dtype)
        except:
            pass
    big_future = client.scatter(data_shards)
    results    = client.map(action,big_future)
    return results


In [38]:
import subprocess

In [66]:
import getpass

def action(data_shard):
    import shapely
    import starepandas
    ret = []
    for d in data_shard:
        if d == 0:
            ret.append((d,'touched',os.getcwd(),os.environ))
        if d == 1:
            ret.append((d,'touched',os.getcwd(),os.listdir()))
        if d == 2:
            ret.append((d,'touched',socket.gethostname()))
        if d == 3:
            filename='.bashrc'
            with open(filename, 'r') as file:
                data = file.read()
            ret.append((d,'read .bashrc',data))
        if d == 4:
            ret.append((d,os.environ['DASK_PARENT'],os.environ['DASK_GATEWAY_WORKER_NAME']))
        if d == 5:
            ret.append((d,pystare.from_latlon([30],[30],4)))
            pass
        if d == 6:
            ret.append((d,os.environ))
            
        # Here's what we really need to work.
        if d == 7:
            client = boto3.client('s3')
            keys = get_s3_keys('daskhub-data',client,prefix='')
            ret.append((d,list(keys)[0:10]))
            
        if d == 8:
            uid = getpass.getuser()
            ret.append((d,uid,))
        
        if d == 9:
            exe = subprocess.Popen(['conda','search','shapely']
                               # ,cwd='/home/jovyan/data/tmp/'
                                   ,stdout=subprocess.PIPE
                                   ,stderr=subprocess.STDOUT
                                  )
            stdout,stderr=exe.communicate()
            # print('vars: ',file_name)
            ret.append((d,"status out=%s\nerr=%s"%(stdout,stderr)))
    return ret
            

In [35]:
if True:
    s3_client = boto3.client('s3')
    keys = get_s3_keys('daskhub-data',s3_client,prefix='')
    pprint.pprint(list(keys)[0:10])

['GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191203.nc4',
 'GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191222.nc4',
 'GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191226.nc4',
 'MODAPS/CATALOGS/MOD05_L2/2019/356/catalogue.sqlite',
 'MODAPS/MOD05/MOD05_L2.A2019336.2315.061.2019337071952.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2315.061.2019337071952_stare.nc',
 'MODAPS/MOD05/MOD05_L2.A2019336.2320.061.2019337072008.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2320.061.2019337072008_stare.nc',
 'MODAPS/MOD05/MOD05_L2.A2019336.2325.061.2019337072403.hdf',
 'MODAPS/MOD05/MOD05_L2.A2019336.2325.061.2019337072403_stare.nc']


In [44]:
data = numpy.arange(10)
data[:] = 3
data[0] = 2 # hostname
data[1] = 0 # basics + os env.
data[2] = 5 # pystare check
data[3] = 8 # getuser
data[4] = 7 # boto3
data[5] = 9 # Popen
data

array([2, 0, 5, 8, 7, 9, 3, 3, 3, 3])

In [68]:
results = slam(client,action,data)

In [69]:
results

[<Future: finished, type: builtins.list, key: action-718cb514259f00a14f5386327c88a892>,
 <Future: finished, type: builtins.list, key: action-157d456eec7bc2259d0adeb82b72c71c>,
 <Future: finished, type: builtins.list, key: action-5c481c6266ffea5e0ee3e7fe5964217b>]

In [70]:
client.gather(results)
# print(results[0].result()[0][2])

[[(2, 'touched', 'dask-worker-6f3d78cc545d4758a4e1c01d3c47c0fb-rrfzp'),
  (0,
   'touched',
   '/home/dask',
   environ{'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
           'HOSTNAME': 'dask-worker-6f3d78cc545d4758a4e1c01d3c47c0fb-rrfzp',
           'DASK_GATEWAY_API_TOKEN': '/etc/dask-credentials/api-token',
           'DASK_GATEWAY_TLS_CERT': '/etc/dask-credentials/dask.crt',
           'DASK_GATEWAY_TLS_KEY': '/etc/dask-credentials/dask.pem',
           'DASK_GATEWAY_WORKER_NAME': 'dask-worker-6f3d78cc545d4758a4e1c01d3c47c0fb-rrfzp',
           'AWS_ROLE_ARN': 'arn:aws:iam::306565948140:role/eksctl-smce-eks-dev-addon-iamserviceaccount-Role1-O3ABBCE1PTBI',
           'AWS_WEB_IDENTITY_TOKEN_FILE': '/var/run/secrets/eks.amazonaws.com/serviceaccount/token',
           'DASK_GATEWAY_API_URL': 'http://api-dhub-dask-gateway.daskhub:8000/api',
           'DASK_GATEWAY_CLUSTER_NAME': 'daskhub.6f3d78cc545d4758a4e1c01d3c47c0fb',
           'PROXY_P

In [17]:
cluster.shutdown()

In [21]:
clusters = gateway.list_clusters(); clusters

[ClusterReport<name=daskhub.fd3a2fb9478a4264b86a60ccd840de8c, status=PENDING>,
 ClusterReport<name=daskhub.286d06b40fc247928b672c51d7b63d09, status=PENDING>]

In [20]:
cluster

In [32]:
clusters


[ClusterReport<name=daskhub.b2d9f9ce32914d2f9cf4af83932d3f66, status=RUNNING>,
 ClusterReport<name=daskhub.b7c48cdfbb234366a318f580154f05a6, status=RUNNING>]

In [55]:
for i in clusters:
    print(i); cluster = gateway.connect(i.name); cluster.shutdown()

ClusterReport<name=daskhub.66c15b6349c44e0fb7e2e117dba88440, status=RUNNING>


In [10]:
cluster = gateway.connect(clusters[0].name)

KeyboardInterrupt: 

In [44]:
cluster.shutdown(); cluster.close()

In [11]:
gateway.close()

In [1]:
gateway.list_clusters()

NameError: name 'gateway' is not defined

In [60]:
gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>

In [59]:
gateway.close()